In [1]:
# Imports

import os
import json
import tqdm
import yaml

import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from omegaconf import OmegaConf
from pathlib import Path
from tqdm.auto import tqdm

/home/denis/miniconda3/envs/uncertainty_evaluation/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash
mkdir -p results
fusermount -uz results
sshfs compute.ies:/mnt/work/dhuseljic/results/uncertainty results
ls results

CIFAR10


In [3]:
def load_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

def load_results(path):
    path = Path(path)
    assert path.is_dir(), 'Path does not exist.'
        
    results = {}

    exp_json = path / 'results_final.json'
    exp_cfg = path / '.hydra' / 'config.yaml'
    try:
        cfg =  OmegaConf.load(exp_cfg)
        data = load_json(exp_json)
    except:
        print(f'{path} has missing results.')
        return

    results['cfg'] = cfg
    # results['checkpoint'] = checkpoint
    results['results'] = data
    return results

def get_experiments(result_path, glob_pattern, train_results=False):
    # Aggregate results over multiple glob pattern such as seeds
    experiments = []
    for exp_path in result_path.glob(glob_pattern):
        d = load_results(exp_path)
        experiments.append(d)
    assert len(experiments) != 0, f'No experiments found for {result_path}.'
    return experiments

path = f'results/CIFAR10/resnet18/100/'
exp_results = get_experiments(Path(path), 'seed1')
# metrics = get_metric_values(exp_results)
print(exp_results[0]['results'].keys())
print(len(exp_results[0]['results']['train_history']))
print(exp_results[0]['results']['test_stats'].keys())

dict_keys(['test_stats', 'train_history', 'test_history', 'misc'])
200
dict_keys(['test_acc1', 'test_prec', 'test_loss', 'test_nll', 'test_tce', 'test_mce', 'test_SVHN_entropy_auroc', 'test_SVHN_conf_auroc', 'test_SVHN_entropy_aupr', 'test_SVHN_conf_aupr'])


In [7]:
dataset = 'CIFAR10'
n_samples = 1000
experiments = {}
experiments.update({
    'Deterministic': f'results/{dataset}/resnet18/{n_samples}',
    'Deterministic Label Smooting alpha=.1': f'results/{dataset}/resnet18_labelsmoothing/{n_samples}',
    'Deterministic Label Smooting alpha=.01': f'results/{dataset}/resnet18_labelsmoothing__alpha=0.01/{n_samples}',
    'Deterministic Label Smooting alpha=.05': f'results/{dataset}/resnet18_labelsmoothing__alpha=0.05/{n_samples}',
})


all_results = {}
for exp_name, exp_path in tqdm(experiments.items()):
    all_results[exp_name] = get_experiments(Path(exp_path), 'seed*')


 50%|█████     | 2/4 [00:00<00:00,  4.61it/s]

results/CIFAR10/resnet18_labelsmoothing/1000/seed3 has missing results.
results/CIFAR10/resnet18_labelsmoothing/1000/seed2 has missing results.
results/CIFAR10/resnet18_labelsmoothing/1000/seed1 has missing results.


100%|██████████| 4/4 [00:00<00:00,  6.10it/s]


In [8]:
result_dict = {}

for exp_name in experiments:
    d = {}
    for metric_key in ['test_acc1', 'test_nll', 'test_tce', 'test_SVHN_entropy_aupr']:
        vals = [seed_results['results']['test_stats'][metric_key] for seed_results in all_results[exp_name]]
        d[metric_key] = np.mean(vals)
    result_dict[exp_name] = d
print(pd.DataFrame(result_dict).T.to_markdown())

TypeError: 'NoneType' object is not subscriptable